In [ ]:
import os

import skimage
import skimage.io as sio
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
# Figure 0, autotomy examples and implied usage rates

folders = ["../assets/u1_m0_100_900", "../assets/u1_m1_100_900", "../assets/u1_m2_100_900", "../assets/u1_m3_100_900"]

solves = [["s600","s800"], ["s100", "s600", "s800"], [], []]
bw = 16
for jj, folder in enumerate(folders):
    list_dir = os.listdir(folder)
    img = None
    for filepath in list_dir:
        if filepath.endswith("png"):
            temp_img = sio.imread(os.path.join(folder, filepath))[100:600, 300:-300]
            temp_img[:bw,:,:] = .0
            temp_img[:,:bw,:] = 0
            temp_img[-bw:,:,:] = 0
            temp_img[:,-bw:,:] = 0
            
            if os.path.splitext(filepath)[0][-4:] in solves[jj]:
                print(f"solve indicated in {filepath}")
                # teal border for solve
                temp_img[:bw,:,1:3] = 255
                temp_img[:,:bw,1:3] = 255
                temp_img[-bw:,:,1:3] = 255
                temp_img[:,-bw:,1:3] = 255
            else:
                # red border for no solve
                temp_img[:bw,:,0] = 255
                temp_img[:,:bw,0] = 255
                temp_img[-bw:,:,0] = 255
                temp_img[:,-bw:,0] = 255

            if img is None:
                img = temp_img
            else:
                img = np.append(img, temp_img, axis=0)
                
    
        
        if img is not None:
            folder_name = os.path.split(folder)[-1]
            sio.imsave(os.path.join("../assets", f"bar_{folder_name}.jpg"), img)
            
        

In [ ]:
# Figure 1 forward/reverse

fig, ax = plt.subplots(1,2, figsize=(10,5))
forward = sio.imread("../assets/forward_square_s2.png")[100:600,300:-300,:]
#forward_frame_agent0_epd0_step24.png")[100:600,300:-300,:]
reverse = sio.imread("../assets/reverse_square_s2.png")[100:600,300:-300,:]
#reverse_frame_agent0_epd0_step1033.png")[:500,300:-300,:]
print(reverse.shape)

ax[0].imshow(forward)
ax[0].set_title("Forward Reward", fontsize=22)
ax[1].imshow(reverse)
ax[1].set_title("Reverse Reward,\n Post-Autotomy", fontsize=22)
ax[1].arrow(300,50,-125,0, color="green", width=20, head_width = 40)

ax[0].arrow(300,50,125,0, color="green", width=20, head_width = 40)

for xx in range(2):
        ax[xx].set_xticklabels("")
        ax[xx].set_yticklabels("")

plt.savefig("../assets/forward_reverse.png")
plt.show()


# draw_neural_net from https://gist.github.com/riveSunder/05c7728c052f2ba28b494ad352f7e939
# adapted from craffel at https://gist.github.com/craffel/2d727968c3aaebd10359

def draw_neural_net(ax, left, right, bottom, top, layers):
    """
 
    Draw a neural network cartoon using matplotilb.
    
    :usage:
        >>> fig = plt.figure(figsize=(12, 12))
        >>> [np.random.randn(12,16),np.random.randn(16,4)]
    
    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here[np.random.randn(12,16),np.random.randn(16,4)]
        - layers : list of weight arrays
    """
    ax.axis('off')
    n_layers = len(layers)
    layer_sizes = [layer.shape[0] for layer in layers]
    layer_sizes.append(layers[-1].shape[1])
    
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color=np.ones(3)*0.9, ec='k', zorder=40)
            ax.add_artist(circle)
    my_cmap = plt.get_cmap("magma")
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                weight_value = layers[n][m,o]
                if weight_value:
                    wv_sign = np.sign(weight_value)
                    my_color = my_cmap(64) if weight_value < 0.0 else my_cmap(192) #'r' if weight_value < 0.0 else 'b'
                    my_color = 'k' if weight_value == 1.0 else my_color
                    line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                      [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], \
                                     lw = 3*np.abs(weight_value), color=my_color, alpha = 0.75)
                    ax.add_artist(line)
                    
fig = plt.figure(figsize=(5,5))
draw_neural_net(fig.gca(), .1, .9, .1, .9, [np.random.randn(4,6),np.random.randn(6,3)])
#plt.savefig("../assets/neural_net.png")
plt.show()



In [ ]:
# Figure 2, body plan modes

# mode 0, joint evo
joint = sio.imread("../assets/joint_frame_agent0_epd0_step0001.png")[:,300:-300,:]
# mode 1, square
square = sio.imread("../assets/square_frame_agent0_epd0_step0001.png")[:,300:-300,:]
# mode 2, table
table = sio.imread("../assets/table_frame_agent0_epd0_step0001.png")[:,300:-300,:]
# mode 3, comb
comb = sio.imread("../assets/comb_frame_agent0_epd0_step0001.png")[:,300:-300,:]

fig, ax = plt.subplots(2,2, figsize=(10,10))

ax[0,0].imshow(joint)
ax[0,0].set_title("Mode 0: Evolved Body Plan", fontsize=20)
ax[0,1].imshow(square)
ax[0,1].set_title("Mode 1: Square Body Plan", fontsize=20)
ax[1,0].imshow(table)
ax[1,0].set_title("Mode 2: Table Body Plan", fontsize=20)
ax[1,1].imshow(comb)
ax[1,1].set_title("Mode 3: Comb Body Plan", fontsize=20)

for xx in range(2):
    for yy in range(2):
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")

#plt.savefig("../assets/body_modes.png")
plt.show()



In [ ]:
#curve_1 = sio.imread("../assets/evolve_autotomy_1progress_BackAndForthEnv-v0_Geneti1665820974_s5.png")
curve_1 = sio.imread("../assets/evolve_autotomy_1progress_BackAndForthEnv-v0_Geneti1665913282_s13.png")
curve_2 = sio.imread("../assets/evolve_autotomy_1progress_BackAndForthEnv-v0_Geneti1665775243_s1.png")

fig, ax = plt.subplots(3,2, figsize=(8,12))

fail_0 = sio.imread("../assets/autotomy_failed_s1_frame0.png")[:,300:-300,:]
fail_1 = sio.imread("../assets/autotomy_failed_s1_frame1.png")[:,300:-300,:]

succeed_0 = sio.imread("../assets/autotomy_succeed_s13_frame0.png")[:,300:-300,:]
succeed_1 = sio.imread("../assets/autotomy_succeed_s13_frame1.png")[:,300:-300,:]

ax[0,0].imshow(succeed_0)
ax[1,0].imshow(succeed_1)

ax[0,1].imshow(fail_0)
ax[1,1].imshow(fail_1)

ax[2,0].imshow(curve_1)
#ax[1,2].imshow(img_2)
ax[2,1].imshow(curve_2)

for xx in range(3):
    for yy in range(2):
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")

plt.tight_layout()
plt.savefig("../assets/autotomy_placeholder.png")
plt.show()

In [ ]:
curve_1 = sio.imread("")
curve_2 = sio.imread("")
curve_2 = sio.imread("")

fig, ax = plt.subplots(3,2, figsize=(8,12))

fail_0 = sio.imread("")[:,300:-300,:]
fail_1 = sio.imread("")[:,300:-300,:]

succeed_0 = sio.imread("")[:,300:-300,:]
succeed_1 = sio.imread("")[:,300:-300,:]

ax[0,0].imshow(succeed_0)
ax[1,0].imshow(succeed_1)

ax[0,1].imshow(fail_0)
ax[1,1].imshow(fail_1)

ax[2,0].imshow(curve_1)
#ax[1,2].imshow(img_2)
ax[2,1].imshow(curve_2)

for xx in range(3):
    for yy in range(2):
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")

plt.tight_layout()
plt.savefig("../assets/nototomy_placeholder.png")
plt.show()